# Lecture 1-2. 구현 실습  

예시 문제 링크 
- [1번: 백준 2578번](https://www.acmicpc.net/problem/2578)
- [2번: 백준 12100번](https://www.acmicpc.net/problem/12100)
- [3번: 코드트리 마법의 숲 탐색](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/magical-forest-exploration/description)
- [4번: 코드트리 청소는 즐거워](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/cleaning-is-joyful/description)
- [5번: 코드트리 미생물 연구](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/microbial-research/description)
- [6번: 코드트리 민트초코 우유](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/mint-choco-milk/description)
- [7번: 코드트리 메두사와 전사들](https://www.codetree.ai/ko/frequent-problems/samsung-sw/problems/medusa-and-warriors/description)


## 3번 문제 해석 


문제 정리

- 격자/좌표: 
  - 숲은 R×C 격자. 문제 기준으로 가장 위가 1행, 가장 아래가 R행(좌표는 1-based). 정령(골렘)은 북쪽(격자 위쪽) 바깥에서 진입한다. 
  - (구현 팁) 진입·회전 동작을 편하게 처리하려고 상단에 3행을 더 붙인 격자(R+3×C) 로 두고 시뮬레이션하는 전형적인 방식이 많이 쓰인다. 
- 입력 & 골렘 정의 (총 K번)
  - 한 번의 시도마다 열 c(1…C), 출구 방향 d(0:북, 1:동, 2:남, 3:서) 가 주어진다. 골렘은 십자(+) 모양(중심+상하좌우) 으로 5칸을 차지하며, 출구는 중심에서 d 방향으로 인접한 1칸이다. 
- 골렘 하강/회전 규칙 (drop)
    - 골렘은 북쪽 밖에서 시작해 아래로 떨어지며 다음 우선순위로 움직인다:
      - 아래로 직하가 가능하면 아래로 1칸, 아니면 왼쪽 회전하며 아래(좌하)(출구는 반시계 90° 회전), 그래도 안 되면 오른쪽 회전하며 아래(우하)(출구는 시계 90° 회전).
      - 세 경우 모두 불가하면 그 자리에서 정지한다. 각 동작은 문제 그림처럼 지정된 검사 칸들이 모두 비어있어야 가능하다. 
- 정착 실패(리셋) 조건
    - 골렘이 멈췄을 때 십자 5칸 중 한 칸이라도 숲 밖(상단 패딩 포함 관점)이라면, 지금까지 놓인 모든 골렘을 전부 지우고 이번 시도는 0점 처리 후 다음 시도로 넘어간다. 
- 정착 성공 시 배치
  - 정지 위치에 중심+팔 4칸을 기록하고, 출구 방향도 함께 저장한다(다음 단계 탐색에 필요). 
- 정령 탐색 규칙 (explore)
    - 시작점: 방금 놓인 골렘의 중심 칸에서 시작. 
    - 이동 방법: 상하좌우로 인접 칸을 보며, 같은 골렘 내부(같은 id)로는 자유롭게 이동 가능,
    - 다른 골렘으로 넘어갈 때는 ‘현재 칸이 출구’일 때만 인접한 그 골렘 칸으로 진입 가능(출구를 통해서만 다른 골렘으로 이동). 
- 점수: 이번 시도에서 정령이 도달할 수 있는 칸들 중 가장 남쪽(행 번호가 가장 큰) 칸의 행 번호(원래 숲 좌표 기준)를 누적 합에 더한다. 구현에서 상단 3행 패딩을 썼다면 반환 전에 그만큼 보정한다. 

- 출력
    - K번 시도 전체에 대해 위 점수를 누적한 총합을 출력한다. 

![3-1](../assets/img/implementationPS/3/1.png)

In the beginning, the golem may extend up to `3 cells` above the forest. To handle this, we add 3 buffer rows at the top of the grid. These buffer rows are not part of the forest but provide enough room for the golem to move and settle safely. 

![3-2](../assets/img/implementationPS/3/2.png)

When the golem moves one step down, we must check 3 positions below the center: directly underneath, and one cell to the left and right. If all 3 are empty, the golem can move downward without collision. 

![3-3](../assets/img/implementationPS/3/3.png)

Now let’s look at rotation to the left. For the golem to rotate, `five specific cells` must be empty. Interestingly, the rules allow rotation even if the upper-right or lower-right cells are occupied. We number the directions North, East, South, and West as 0, 1, 2, and 3. Using this, left rotation is simply (d + 3) mod 4, and right rotation is (d + 1) mod 4.

![3-4](../assets/img/implementationPS/3/4.png)

Rotation to the right works symmetrically. Again, five surrounding cells must be empty to allow the move. The concept is the same as left rotation, but we apply the clockwise formula.

![3-5](../assets/img/implementationPS/3/5.png)

When the golem finally stops, all four arms must be inside the forest area, which is from row 3 down to H–1. To manage state, we store each golem’s position and exit direction in units[gid]. We also record which exit belongs to which golem in exit_map. This way, exit_map tells us the gid, and units gives us the exact center and direction.

![3-6](../assets/img/implementationPS/3/6.png)

We maintain two separate grids: `golem_arr` and `exit_map`. The `golem array` records body occupancy and is used for `collision checks`. The `exit map` records `only exits`, which lets us trace connectivity between golems using `BFS`. This separation is crucial because body cells and exit cells need to be treated differently.

![3-7](../assets/img/implementationPS/3/7.png)

Finally, let’s look at how we propagate the maximum reachable row. 
- Step one: if `the current golem’s exit` touches a neighbor’s body, we inherit its `max_row` as the `seed` value. 
- Step two: using BFS, we spread values along exit-to-exit connections across the whole component. 

This two-step design ensures accuracy — newly placed golems immediately get the correct value. It also improves efficiency, since we avoid repeated updates by maximizing early and propagating just once.

### Keys to the Solution 

- The key idea is simulation + connectivity propagation.
- Forest size: R × C, with K golems.
- Each golem has a cross (+) shape: the center plus 1 cell in each of the four directions (total of `5 cells`). Its exit is the cell adjacent to the center in the facing direction.
- A golem falls from above and attempts to move in the following order:
  - Straight down
  - Down-left (rotate exit to the left)
  - Down-right (rotate exit to the right)
  - If all three moves fail, it stops.
- To simplify the simulation, the board is `extended with 3 extra rows` at the top (so the grid size is (R+3)xC)
- After `settling`, if any of the 4 arms of the golem lie `outside the original forest` (rows indexed 3 to R+2), the entire forest is `reset` (all cleared) and that golem is `ignored`.
- In other words, after placement, the body of the golem (the four arms of the cross) must be fully contained within the forest.
- Once a golem is placed, `compute the maximum reachable row` (in the original forest, indices 0 to R−1) within the connected component that includes this golem and all others connected via exits. Add this value to the final sum.

## 5번 문제 해석

문제 정리

1. NxN 정사각형 배양 용기, 좌측 하단 (0,0), 우측 상단 (N, N)
2. 총 Q번 실험동안, 실험 결과 기록 
    - 2-1. insert_and_get_result() 미생물 투입 후, 배양 
        - (r1, c1) ~ (r2, c2) 직사각형 미생물 투입, 영역내에는 새로 투입된 미생물만 남음. 
        - 기존에 있던 미생물이 먹힌 경우, 영역이 두 개 이상으로 나뉘면 기존 미생물은 모두 사라짐.  -> 항상 미생물은 하나의 그룹으로만 이루어지게 만듦. 
    - 2-2. 배양 용기 이동: move_microbe()
        - 기존 용기와 동일한 새로운 배양 용기로 이동. 기존 배양 용기에 미생물이 한 마디도 남지 않을 때까지 
            - 가장 차지한 영역 ((r1, c1) ~ (r2, c2))이 넓은 무리 하나 선택 -> 2개 이상이면 먼저 투입된 미생물 선택 (용기에는 미생물을 넣은 시간 정보 필요 -> idx가 작은 것 선택)
            - 선택된 미생물의 영역 ((r1, c1) ~ (r2, c2))은 
                - 배양 용기의 범위를 벗어나지 않으며 
                - 다른 미생물의 영역과 겹치지 않아야하고, 
                - 위의 조건에서 최대한 x좌표가 작은 위치로 미생물을 옮기는데, 
                    - 그 위치가 2개 이상이면 최대한 y좌표가 작은 위치로 오도록 옮김 
                - 위의 조건이 없다면, 옮기지 못하고 사라짐 
    - 2-3. record() 
        - 미생물 무리 중 상하좌우로 닿아있는 인접한 무리 확인 
        - 맞닿은 면이 둘 이상이더라도 micro_id같으면 한 번만 확인 
        - 인접한 무리가 있으면 
            - 확인하는 두 무리가 A,B라면 (미생물 A영역의 넓이) x (미생물 B영역의 넓이) 성과 
            - 확인한 모든 쌍의 성과 기록 

# 정답 코드

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Daye-Lee18/mybook/blob/main/assets/ipynb/1_implementationPS.ipynb
)